In [1]:
import torch
import pickle

import sys
sys.path.append('/home/ql2221/Projects/thermalizer/thermalizer/models')
import unet_modern as munet
sys.path.remove('/home/ql2221/Projects/thermalizer/thermalizer/models')

sys.path.append('/home/ql2221/Projects/Joan_Bruna_work/UUnet/Models')
import Unet as Unet

In [2]:
#get the emulator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

file_string = '/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250526_223850-r12kgbg1/files/checkpoint_best.p'
with open(file_string, 'rb') as f:
    emulator_checkpoint = pickle.load(f)

# Extract the configuration (this might include hyperparameters, model architecture details, etc.)
emulator_config = emulator_checkpoint['config']

if emulator_config["model_type"]=="ModernUnet":
    emulator = munet.ModernUnet(emulator_config).to(device)

emulator.load_state_dict(emulator_checkpoint["state_dict"])

<All keys matched successfully>

In [3]:
checkpoint_string = "/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250619_023225-xmbfc9iy/files/checkpoint_last.p"
with open(checkpoint_string, 'rb') as f:
    flow_matching_checkpoint = pickle.load(f)

flow_matching_config = flow_matching_checkpoint['config']

if flow_matching_config["model_type"]=="ModernUnet":
    flow_matching = Unet.ModernUnet(flow_matching_config).to(device)

flow_matching.load_state_dict(flow_matching_checkpoint["state_dict"])

<All keys matched successfully>

In [4]:
#generate an emulator rollout, and denoise every step using flow matching
sys.path.append('/home/ql2221/Projects/Joan_Bruna_work/Flow_matching/Inference')
import Roll_out_with_flow_matching as Flow

In [5]:
#load data
loaded_data = torch.load("/scratch/ql2221/thermalizer_data/kolmogorov/reynold10k/test_long.p")
data = loaded_data["data"]

x = data[0,:,:,:]
x = x.unsqueeze(0).to(device)
print(x.shape)
x = x/4.44

torch.Size([1, 90000, 64, 64])


In [6]:
rollout = Flow.run_flow_matching_emu(x[:,0:1], emulator, flow_matching, 1000)

In [7]:
import matplotlib.pyplot as plt
from matplotlib import animation
def make_movie(state_vector, save_path="emulator_movie.mp4", fps=30, vmin=None, vmax=None, stride=10):
    """
    Create a movie from a 4D tensor of shape (batch, time, height, width), using only every `stride`-th frame.
    
    Parameters:
    - state_vector: torch.Tensor of shape (B, T, H, W)
    - save_path: output video file path
    - fps: frames per second for the movie
    - vmin, vmax: color scale
    - stride: only include every `stride`th frame in the video
    """
    B, T, H, W = state_vector.shape
    data = state_vector[0].cpu().numpy()

    fig, ax = plt.subplots()
    im = ax.imshow(data[0], cmap="viridis", vmin=vmin, vmax=vmax)
    ax.set_title("Timestep 0")
    fig.colorbar(im, ax=ax)

    # Frame indices to use
    frame_indices = list(range(0, T, stride))

    def update(frame_idx):
        frame = frame_indices[frame_idx]
        im.set_data(data[frame])
        ax.set_title(f"Timestep {frame}")
        return im,

    ani = animation.FuncAnimation(fig, update, frames=len(frame_indices), blit=False, repeat=False)
    ani.save(save_path, fps=fps)
    plt.close(fig)
    print("movie saved")

In [8]:
rollout = rollout[0]
make_movie(rollout, save_path="rollout.mp4", fps=30)

movie saved
